In [0]:
%md
s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/
s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date={}/

In [0]:
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd

pd.set_option('expand_frame_repr', False)
import bo
s3 = boto3.resource('s3')
s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/gameiq/2020-03-13/regression5.txt')
log = [] 

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/_partition_col=0/

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/ | tail -5

In [0]:
%%sh
# date=2010-07-04/--date=2020-04-04/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/ | tail -4


In [0]:
%%sh


In [0]:

from pyspark.sql import functions as F
df_dna = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").withColumn("genre_id", F.explode("genre_id"))
df_dna.show(3)

df_all_genre = df_dna.groupby("genre_id").agg({'genre_id': 'count'})
print len(df_all_genre.collect())

In [0]:

df_store_app = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-02-29/")
df_store_app.show(3)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date=2016-07-24/

In [0]:

# All modifier_id is 100000
df_store_genre = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date=2020-01-29/")
# df_store_genre = df_store_genre.groupBy("modifier_id").agg({"modifier_id": "count"})
# android-all|          AE|  303006|     171|    877|
df_store_genre = df_store_genre.where("device_code='android-all' and country_code='AE' and genre_id='303006'")
df_store_genre.show(3)

In [0]:
%%sh
aws s3 cp s3://b2c-prod-data-pipeline-qa/tom/gameiq/regression5.txt - 

In [0]:


import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df



DEVICE_CODE_MAPPING_BY_DEVICE_ID = {
    'google-play':{
        1000: "android-all",
        1001: "android-phone",
        1002: "android-tablet",
    },
    'apple-store': {
        2000: "ios-all",
        2001: "ios-phone",
        2002: "ios-tablet"
    }
}
DEVICE_CODE_MAPPING = DEVICE_CODE_MAPPING_BY_DEVICE_ID

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)

mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/")
df_pid2gid = mapping_df_unified.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id'))

def compare(date):
    #collect
    df_store_app_est = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))
    est_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date={}/".format(date))
    
    # #transform
    df_store_app_est_trans = df_store_app_est.select(["app_id", "device_code", "country_code", "free_app_download", "paid_app_download", "revenue"]).withColumnRenamed("app_id", "product_id")
    # df_store_app_est_trans.show(1)
    # df_pid2gid.show(3)
    # print 'after join:'
    giq_df = df_store_app_est_trans.join(df_pid2gid, df_pid2gid.product_id == df_store_app_est_trans.product_id, how='inner').groupBy(["device_code", "country_code", "genre_id"]).agg({
            "free_app_download": "sum",
            "paid_app_download": "sum",
            "revenue": "sum",
        }).withColumnRenamed("sum(free_app_download)", "free_app_download").withColumnRenamed("sum(paid_app_download)", "paid_app_download").withColumnRenamed("sum(revenue)", "revenue")
    # giq_df.show(1)
    giq_df = giq_df.fillna({"free_app_download":0, "paid_app_download":0})
    giq_df = giq_df.withColumn("download", F.when(giq_df.free_app_download + giq_df.paid_app_download>0, giq_df.free_app_download + giq_df.paid_app_download).otherwise(F.lit(None)))

    #compare
    s1=giq_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])
    # s1.show(1)

    s2=est_unified_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])
    # s2.show(1)

    diff = s1.union(s2).subtract(s1.intersect(s2))
    
    ##########################
    # DB & UNIFIED LAYER TEST

    sql = """
    select device_id, store_id, date, genre_id, modifier_id, download, revenue 
    from plproxy.execute_select_nestloop($proxy$ 
    select device_id, store_id, date, genre_id, modifier_id, download, revenue
    from aa.genre_store_daily_estimate
    where date = '{}'
    $proxy$) tbl (device_id SMALLINT, store_id INT, date DATE , genre_id BIGINT, modifier_id BIGINT, download BIGINT, revenue BIGINT);""".format(date)

    # db_df = query_df(daily_est_dsn, sql)

    # db_df.loc[db_df['device_id'].isin(DEVICE_CODE_MAPPING['google-play'])] = db_df.loc[db_df['device_id'].isin(DEVICE_CODE_MAPPING['google-play'])].replace({"device_id": DEVICE_CODE_MAPPING['google-play']})
    # db_df.loc[db_df['device_id'].isin(DEVICE_CODE_MAPPING['apple-store'])] = db_df.loc[db_df['device_id'].isin(DEVICE_CODE_MAPPING['apple-store'])].replace({"device_id": DEVICE_CODE_MAPPING['apple-store']})

    # db_df = db_df.replace({"store_id": COUNTRY_CODE_MAPPING['google-play']})
    # db_df = db_df.replace({"store_id": COUNTRY_CODE_MAPPING['apple-store']})
    
    # db_df = db_df.rename(columns={'store_id': 'country_code'}).rename(columns={'device_id': 'device_code'}).rename(columns={'device_id': 'device_code'}).rename(columns={'device_id': 'device_code'})
    
    # est_unified_df = est_unified_df.toPandas()
    # est_unified_df["modifier_id"] = 100000

    # diff_db = _compare_df(est_unified_df, db_df)
    
    if diff.count()>0:
        print "diff:"
        diff.show(2)
        print "diff groupby country:"
        country_group = diff.groupby("country_code").agg({"country_code": "count"})
        return ["{}: FAIL UNIFIED, Diff length: {}".format(date, diff.distinct().count()), country_group]
    if len(diff_db)>0:
        print diff_db
        return "{}: FAIL DB".format(date)
    return  "{}: PASS".format(date)
    
    
def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

def _compare_df(df1, df2):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type)  # .loc[lambda x : x['_merge']!='both']
        diff_df = diff_df.loc[diff_df["_merge"] != "both"]
        if len(diff_df) != 0:
            print diff_type
            return diff_df
    return []

def write_log(strobj, s3obj):
    s3obj.put(Body=str(strobj))

date_list = get_date_list("2020-04-04", "2020-04-04")
# date_list = get_date_list("2018-01-01/", "2020-02-29/")
country_group_list = []
for date in date_list:
    temp_log = []
    try:
        temp_log = compare(date)
    except Exception, e:
        temp_log = "{}: ERROR".format(date) 
    log.append(temp_log)
    print temp_log[0]
    country_group_list.append(temp_log[1])
    # write_log(log, s3object)

# for country_group in country_group_list:
#     print "country group count:", country_group.count()


In [0]:

country_group_list

In [0]:

country_group_list[0].subtract(country_group_list[1]).show()
    

In [0]:

mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/")
df_pid2gid = mapping_df_unified.select(["product_id", "genre_id"]).withColumn('genre_id', F.explode('genre_id'))

date = '2020-01-29'
#collect
df_store_app_est = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date={}/".format(date))
est_unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity=daily/date={}/".format(date))

# #transform
df_store_app_est_trans = df_store_app_est.select(["app_id", "device_code", "country_code", "free_app_download", "paid_app_download", "revenue"]).withColumnRenamed("app_id", "product_id")
df_store_app_est_trans.show(1)
df_pid2gid.show(3)
print 'after join:'
giq_df = df_store_app_est_trans.join(df_pid2gid, df_pid2gid.product_id == df_store_app_est_trans.product_id, how='inner').groupBy(["device_code", "country_code", "genre_id"]).agg({
        "free_app_download": "sum",
        "paid_app_download": "sum",
        "revenue": "sum",
    }).withColumnRenamed("sum(free_app_download)", "free_app_download").withColumnRenamed("sum(paid_app_download)", "paid_app_download").withColumnRenamed("sum(revenue)", "revenue")
giq_df.show(3)
giq_df = giq_df.fillna({"free_app_download":0, "paid_app_download":0})
giq_df = giq_df.withColumn("download", F.when(giq_df.free_app_download + giq_df.paid_app_download>0, giq_df.free_app_download + giq_df.paid_app_download).otherwise(F.lit(None)))

#compare
s1=giq_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])
print "s1:"
s1.show(2)
s2=est_unified_df.select(["device_code", "country_code", "genre_id", "download", "revenue"])
print "s2:"
s2.show(2)
diff = s1.union(s2).subtract(s1.intersect(s2))
diff.show(3)

In [0]:

# android-all|          AE|  303006|     171|    877|
s1 = s1.where("device_code='android-all' and country_code='AE' and genre_id='303006'")
s1.show(3)

In [0]:

diff.groupby("country_code").agg({"country_code": "count"}).count()

In [0]:

diff.count()

In [0]:

s1.count()

In [0]:

s2.count()

In [0]:

my_diff = s1.subtract(s2)
my_diff.count()

In [0]:

my_diff2 = s2.subtract(s1)
my_diff2.count()

In [0]:

my_diff.show(10)

In [0]:

my_diff2.show(10)

In [0]:

aaa = my_diff.intersect(my_diff2)
aaa.show(3)

In [0]:

import pandas as pd
df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
     index=['cobra', 'viper', 'sidewinder'],
     columns=['max_speed', 'shield'])
df

In [0]:

print df.loc[[True, True, False]]

In [0]:

df_dna_mapping = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").withColumn("genre_id", F.explode("genre_id"))
df_dna_mapping.show(3)

In [0]:

granularity, date = 'daily', '2019-02-10'
df_store_genre = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity={}/date={}/".format(granularity, date)).where("genre_id='302006'")
df_store_genre.show(3)

In [0]:

df_all_genre = df_dna_mapping.groupby("genre_id").agg({'genre_id': 'count'})
print df_all_genre.count()
df_all_genre.show(3)

In [0]:

df_all_genre = df_all_genre.select('genre_id')
df_store_genre = df_store_genre.select('genre_id')
diff_genre = df_store_genre.union(df_all_genre).subtract(df_store_genre.intersect(df_all_genre))
diff_genre.show()

In [0]:

df_all_genre.subtract(df_store_genre).show()

In [0]:

import pandas as pd
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list

def compare_genre(granularity, date):
    df_store_genre = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.genre-est.v1/fact/granularity={}/date={}/".format(granularity, date))
    df_dna_mapping = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").withColumn("genre_id", F.explode("genre_id"))
    
    df_all_genre = df_dna_mapping.groupby("genre_id").agg({'genre_id': 'count'})
    
    df_all_genre = df_all_genre.select('genre_id')
    df_store_genre = df_store_genre.select('genre_id')
    
    # diff_genre = df_store_genre.union(df_all_genre).subtract(df_store_genre.intersect(df_all_genre))
    # diff_genre.show()
    
    # diff_genre = df_all_genre.subtract(df_store_genre)
    diff_genre = df_store_genre.subtract(df_all_genre)
    # diff_genre.show()
    return diff_genre

# date=2010-07-04/--date=2020-04-04/
date_list = get_date_list("2018-01-01", "2020-02-29")
# date_list = get_date_list("2020-02-10", "2020-02-11")
print "Data length: ", len(date_list)
log = []

schema = StructType([
    StructField("genre_id", IntegerType(), False)
])
df = spark.createDataFrame([], schema)

for date in date_list:
    res = compare_genre('daily', date) # :type res: list
    if res.take(1):
        # log.append((date, res))
        log.append(date)
        # df = log[0][1]
        # for date, temp_df in log:
        df = df.union(res).distinct()

print "df:"
df.show()

# log  = [date for date, _ in log]
print log

In [0]:


from conf.settings import PG_DNA_NAME, PG_DNA_ACCESS_ID, PG_DNA_HOSTS, PG_DNA_SECRET_KEY
from applications.db_check_v1.common.db_check_utils import query_df
from pyspark.sql.functions import explode

dna_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DNA_NAME,
        user=PG_DNA_ACCESS_ID,
        host=PG_DNA_HOSTS[0][0],
        password=PG_DNA_SECRET_KEY,
        port=PG_DNA_HOSTS[0][1]
    )
)
# mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").toPandas()
sql = 'SELECT genre_id FROM dna_genre_id_product_mapping'
mapping_df_raw = query_df(dna_dsn, sql)
spark_df = spark.createDataFrame(mapping_df_raw)
# mapping_df_raw = mapping_df_raw.explode('genre_id')
exploded_spark_df = spark_df.select(explode('genre_id').alias('genre_id'))
genre_count = exploded_spark_df.distinct().count()
print mapping_df_raw
mapping_df_raw['created_time'] = mapping_df_raw['created_time'].dt.strftime('%Y-%m-%d')
mapping_df_raw['last_updated_time'] = mapping_df_raw['last_updated_time'].dt.strftime('%Y-%m-%d')
mapping_df_raw['genre_id'] = [str(map(int, l))  if l else 'None' for l in mapping_df_raw['genre_id']]
mapping_df_raw['modifier_id'] = [str(map(int, l)) if l else 'None' for l in mapping_df_raw['modifier_id']]

# mapping_df_unified['genre_id'] = mapping_df_unified['genre_id'].astype("str")
# mapping_df_unified['modifier_id'] = mapping_df_unified['modifier_id'].astype("str")
# print mapping_df_raw
# # print mapping_df_unified

# def _compare_df(df1, df2, on=None):
#     for diff_type in ["left", "right"]:
#         diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
#         diff_df = diff_df.loc[diff_df["_merge"] != "both"]
#         print diff_df.empty
#         if len(diff_df) != 0:
#             print diff_type
#             print diff_df


# # # print mapping_df_raw.genre_id.dtypes
# # # print '*'*100
# # # print mapping_df_unified.genre_id.dtypes
# # # print mapping_df_raw
# # # print mapping_df_unified

# _compare_df(mapping_df_raw, mapping_df_unified, on=["product_id", "genre_id"]) # "modifier_id","last_updated_time",  "created_time", "created_by", "last_updated_by", "comments", 
# print "pass"

In [0]:

print genre_count

In [0]:

import pandas as pd
pd.show_versions()

In [0]:

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("k", StringType(), True), StructField("v", IntegerType(), False)
])

# or df = sc.parallelize([]).toDF(schema)

df = spark.createDataFrame([], schema)

In [0]:

df.show()

In [0]:

from pyspark.sql import Row
temp_df = spark.createDataFrame([Row(genre_id=0)])
temp_df.show()
temp_df = temp_df.union(spark.createDataFrame([Row(genre_id=302006), Row(genre_id=301007), Row(genre_id=207003), Row(genre_id=207006)]))
temp_df.union(spark.createDataFrame([Row(genre_id=302006), Row(genre_id=301007), Row(genre_id=207003), Row(genre_id=207006)])).distinct().show()

In [0]:

len(log)

In [0]:

from pyspark.sql import Row
df = spark.createDataFrame([Row(genre_id=0)])
for date, row_list in log[:100]:
    temp_df = spark.createDataFrame(row_list)
    df = df.union(temp_df).distinct()

df.show()


In [0]:

from pyspark.sql import Row
df = spark.createDataFrame([Row(genre_id=0)])
for date, row_list in log[101:200]:
    temp_df = spark.createDataFrame(row_list)
    df = df.union(temp_df).distinct()
df.show()

In [0]:

print log

In [0]:

import unittest
import re
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df
from aadatapipelinecore.core.utils.spark import create_spark
# from conf.settings import PG_DNA_NAME, PG_DNA_ACCESS_ID, PG_DNA_HOSTS, PG_DNA_SECRET_KEY


from conf import settings
def pg_settings(schema):
    template = "PG_" + schema.upper() + "_{property}"
    host, port = getattr(settings, template.format(property='HOSTS'))[0]
    return (
        "dbname='{db}' user='{user}' password='{password}' "
        "host='{host}' port='{port}'".format(
            db=getattr(settings, template.format(property='NAME')),
            user=getattr(settings, template.format(property='ACCESS_ID')),
            host=host,
            password=getattr(settings, template.format(property='SECRET_KEY')),
            port=port
        )
    )

class GenreEstUnifiedData(object):
    """
    Get unified data
    """
    _s3_bucket_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
                      "store.genre-est.v1/fact/granularity=daily/date={}/"

    def __init__(self, spark):
        self.spark = spark

    def get(self, date):
        return self.spark.read.parquet(self._s3_bucket_path.format(date))


class DNAGenreIdProductMappingData(object):
    """
    Get mapping data (DNA) from unified level
    """
    _s3_bucket_path = "s3://b2c-prod-data-pipeline-unified-dna/unified/" \
                      "dna.genre_id_product_mapping.v1/dimension/"

    def __init__(self, spark):
        self.spark = spark

    def get(self):
        return self.spark.read.parquet(self._s3_bucket_path)


class DNAMappingDBData(object):
    """
    Get mapping data (DNA) from DB
    """
    _pg_dsn = pg_settings('DNA')

    def get(self, sql):
        """
        :param sql: sql str
        :return: mapping data
        :rtype: pandas dataframe
        """
        return query_df(self._pg_dsn, sql)


def get_date_list(start_date, end_date, freq="D"):
    return [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]


def compare_df(df1, df2, on=None):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
        diff_df = diff_df.loc[diff_df["_merge"] != "both"]
        if len(diff_df) != 0:
            return diff_type, diff_df
        return None, None

class TestGameIQMarketSizeFullRestatement(PipelineTest):
    trigger_date_config = ('* * * * *', 0)

    @classmethod
    def setUpClass(cls):
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext

    def _compare_genre(self, date):
        """
        :param date: date need to compare
        :return: genre found in store_genre, but not in dna_mapping
        :rtype diff_genre: spark.Dataframe
        """
        df_store_genre = GenreEstUnifiedData(self.spark).get(date)
        df_dna_mapping = DNAGenreIdProductMappingData(self.spark).get()
        df_dna_mapping = df_dna_mapping.withColumn("genre_id", F.explode("genre_id"))

        df_all_genre = df_dna_mapping.groupby("genre_id").agg({'genre_id': 'count'}).select('genre_id')
        df_store_genre = df_store_genre.select('genre_id')
        # diff_genre = df_all_genre.subtract(df_store_genre)
        diff_genre = df_store_genre.subtract(df_all_genre)

        return diff_genre

    # def test_game_iq_market_size_full_restatement(self):
    #     # date_list = get_date_list("2019-07-04", self.check_date_str)
    #     date_list = get_date_list("2020-02-10", "2020-02-11")
    #     log = []
    #     schema = StructType([
    #         StructField("genre_id", IntegerType(), False)
    #     ])
    #     unexpected_df = self.spark.createDataFrame([], schema)  # Create a empty spark.Dataframe
    #     for date in date_list:
    #         try:
    #             res = self._compare_genre(date)
    #             if res.take(1):
    #                 log.append(date)
    #                 # unexpected_df = unexpected_df.union(res).distinct()
    #         except Exception as e:
    #             if not re.search(r'Path does not exist:', str(e)):  # skip date that have no data.
    #                 print e

    #     self.assertEqual(log, [], msg="Unexpected genre {} in {}".format(
    #         [row['genre_id'] for row in unexpected_df.collect()], log
    #     ))

unittest.main(argv=[''], verbosity=2)
%tb

In [0]:

from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df, query
import pandas as pd
pd.set_option('expand_frame_repr', False)

date = "2020-01-01"
sql = """
select device_id, store_id, date, genre_id, modifier_id, download, revenue from plproxy.execute_select_nestloop($proxy$ 
    select device_id, store_id, date, genre_id, modifier_id, download, revenue
    from aa.genre_store_daily_estimate
    where 
        date = '{}'
$proxy$) tbl (device_id SMALLINT, store_id INT, date DATE , genre_id BIGINT, modifier_id BIGINT, download BIGINT, revenue BIGINT);

""".format(date)
daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)

db_df = query_df(daily_est_dsn, sql)
db_data = query(daily_est_dsn, sql)

db_genre = set([])
for row in db_data:
    db_genre.add(row[3])
print len(db_genre), db_genre

# db_df = db_df.loc[:, 'genre_id']
# db_set = set(db_df.values.tolist())
# print db_set

In [0]:

mapping_data = DNAGenreIdProductMappingData(spark).get()

In [0]:

mapping_data = DNAGenreIdProductMappingData(spark).get()
mapping_data = mapping_data.withColumn('genre_id', F.explode('genre_id')).select('genre_id').groupby('genre_id').count().select('genre_id').toPandas()
# mapping_data = mapping_data.loc[:, 'genre_id']
# all_mapping_data = set(mapping_data.values.tolist())
# print all_mapping_data
print mapping_data

In [0]:

float(len(all_mapping_data - db_set)) / float(len(all_mapping_data))

In [0]:

len(all_mapping_data)

In [0]:

# ALL MAPPING DATA
mapping_data = DNAGenreIdProductMappingData(spark).get()
mapping_data = mapping_data.withColumn('genre_id', F.explode('genre_id')).select('genre_id').groupby('genre_id').count().select('genre_id').toPandas().loc[:, 'genre_id']
all_mapping_data = set(mapping_data.values.tolist())
print all_mapping_data


In [0]:

from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df, query
import pandas as pd

sql = """
select device_id, store_id, date, genre_id, modifier_id, download, revenue from plproxy.execute_select_nestloop($proxy$ 
    select device_id, store_id, date, genre_id, modifier_id, download, revenue
    from aa.genre_store_daily_estimate
    where date = '{}'
$proxy$) tbl (device_id SMALLINT, store_id INT, date DATE , genre_id BIGINT, modifier_id BIGINT, download BIGINT, revenue BIGINT);
"""
daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    return [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]

def get_date_range(s_str, e_str, freq="Y"):
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    s = s_str.split('-')
    e = e_str.split('-')
    date_list = []
    start_date = datetime(int(s[0]), int(s[1]), int(s[2]))
    end_date = datetime(int(e[0]), int(e[1]), int(e[2]))
    if freq == "Y":
        while start_date <= end_date:
            date_list.append(start_date.strftime("%Y-%m-%d"))
            start_date += relativedelta(years=+1)
    elif freq == "M":
        while start_date <= end_date:
            date_list.append(start_date.strftime("%Y-%m-%d"))
            start_date += relativedelta(months=+1)
    elif freq == "D":
        while start_date <= end_date:
            date_list.append(start_date.strftime("%Y-%m-%d"))
            start_date += relativedelta(days=+1)
    else:
        raise TypeError("freq must in ['Y', 'M', 'D']")

    return date_list


def temp_log_to_s3(log, name):
    import boto3
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/gameiq/{}.txt'.format(name))
    log_str = ""
    for k, v in log.items():
        log_str += k + ',' + str(v) + ';'
    log_str = log_str[:-1]
    s3object.put(Body=str(log_str))

    return log_str

def s3_to_temp_log(name):
    import boto3
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/gameiq/{}.txt'.format(name))
    body = s3object.get()['Body'].read()
    res = body.split(';')
    res = [i.split(',') for i in res]
    res = {i: int(j) for i, j in res}

    return res


def get_date_and_count(date_list):
    # all_mapping_data_length = float(len(all_mapping_data))
    res = {}
    for date in date_list:
        db_data = query(daily_est_dsn, sql.format(date))
        if not db_data:
            continue
        db_set = set([])
        for row in db_data:
            db_set.add(row[3])
    
        # diff_len = float(len(all_mapping_data - db_set))
        # ratio = diff_len / all_mapping_data_length
        # print date, ratio
        # if ratio > 0.15:
        db_set_len = len(db_set)
        # if db_set_len <= 46 - 20 or db_set_len > 46:
        res[date[:4]] =  db_set_len
    return res
    


SPECIAL_DATE = ['2010-07-04', 37]
date_list = get_date_range('2010-01-01', '2020-04-16', freq='Y')
min_genre_count = get_date_and_count(date_list)
min_genre_count[SPECIAL_DATE[0][:4]] = SPECIAL_DATE[1]
print "min_genre_count:", min_genre_count
temp_log_to_s3(min_genre_count, 'genre_min_count')

date_list = get_date_range('2010-12-31', '2020-04-16', freq='Y')
max_genre_count = get_date_and_count(date_list)
print "max_genre_count:", max_genre_count
temp_log_to_s3(max_genre_count, 'genre_max_count')
# min_genre_count = {'2020': 122, '2019': 123, '2018': 119, '2015': 108, '2014': 98, '2017': 115, '2016': 111, '2011': 46, '2010': 37, '2013': 90, '2012': 76}
# Above code gets all year's min genre count

# Next, I need to verify each day from 2010-07-04 to 2020-04-04
def verify_each_day(date_list, min_genre_count, max_genre_count, threshold=30):
    for date in date_list:
        db_data = query(daily_est_dsn, sql.format(date))
        if not db_data:
            continue
        db_set = set([])
        for row in db_data:
            db_set.add(row[3])  # genre_id is the 3rd column
        db_set_len = len(db_set)
        lower_limit = min_genre_count[date[:4]]
        if date[:4] in max_genre_count:
            upper_limit = max_genre_count[date[:4]]
            if db_set_len > upper_limit:
                print "out of UPPER limit..."
                print date, "genre id count:", db_set_len, "expected count: less than", upper_limit
        else:
            if db_set_len > lower_limit + threshold:
                print "out of UPPER limit...( > lower + {} )".format(threshold)
                print date, "genre id count:", db_set_len, "expected count: less than", lower_limit + threshold
        if db_set_len < lower_limit - 5:
            print "out of LOWER limit..."
            print date, "genre id count:", db_set_len, "expected count: greater than", lower_limit - 5

date_list = get_date_range('2020-04-01', '2020-04-17', freq='D')
verify_each_day(date_list, s3_to_temp_log('genre_min_count'), s3_to_temp_log('genre_max_count'))
print "END"


In [0]:


def get_date_range(s_str, e_str, freq="Y"):
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    s = s_str.split('-')
    e = e_str.split('-')
    date_list = []
    start_date = datetime(int(s[0]), int(s[1]), int(s[2]))
    end_date = datetime(int(e[0]), int(e[1]), int(e[2]))
    if freq == "Y":
        while start_date <= end_date:
            date_list.append(start_date.strftime("%Y-%m-%d"))
            start_date += relativedelta(years=+1)
    elif freq == "M":
        while start_date <= end_date:
            date_list.append(start_date.strftime("%Y-%m-%d"))
            start_date += relativedelta(months=+1)
    elif freq == "D":
        while start_date <= end_date:
            date_list.append(start_date.strftime("%Y-%m-%d"))
            start_date += relativedelta(days=+1)
    else:
        raise TypeError("freq must in ['Y', 'M', 'D']")
    return date_list

# print get_date_range('2010-01-10', '2010-01-12', freq='M')


log = {'2020': 122, '2019': 123, '2018': 119, '2015': 108, '2014': 98, '2017': 115, '2016': 111, '2011': 46, '2010': 37, '2013': 90, '2012': 76}
def temp_log_to_s3(log, name):
    import boto3
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/gameiq/{}.txt'.format(name))
    log_str = ""
    for k, v in log.items():
        log_str += k + ',' + str(v) + ';'
    log_str = log_str[:-1]
    s3object.put(Body=str(log_str))
    return log_str

def s3_to_temp_log(name):
    import boto3
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/gameiq/{}.txt'.format(name))
    body = s3object.get()['Body'].read()
    res = body.split(';')
    res = [i.split(',') for i in res]
    res = {i: int(j) for i, j in res}

    return res

# print temp_log_to_s3(log, 'genre_max_count')
print s3_to_temp_log('genre_max_count')

In [0]:
%%sh
aws s3 cp s3://b2c-prod-data-pipeline-qa/aa.store.game_iq/genre_min_count.txt -


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
select * from dna_genre_id_product_mapping limit 3;
-- select count(1) from ( SELECT distinct unnest(genre_id) FROM dna_genre_id_product_mapping ) as prod;
EOF

In [0]:
%%sh

#\d+ aa.genre_store_daily_estimate_1000_2020


PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
-- \d+ aa.genre_store_daily_estimate;

select distinct date, genre_id from plproxy.execute_select_nestloop(\$proxy\$ 
    select distinct date, genre_id 
    from aa.genre_store_daily_estimate
    where date = '2020-07-03'
    LIMIT 5;
\$proxy\$) tbl (date DATE, genre_id BIGINT)
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p 7432 << EOF 
select date,count(distinct(genre_id)) from plproxy.execute_select_nestloop(\$proxy\$ 
    select date,genre_id
    from aa.genre_store_daily_estimate
    where 
        date between '2019-01-01' and '2019-05-31'
    group by date, genre_id
\$proxy\$) tbl ( date DATE, genre_id BIGINT) group by date;
EOF

In [0]:

# ALL MAPPING DATA
mapping_data = DNAGenreIdProductMappingData(spark).get()
mapping_data = mapping_data.withColumn('genre_id', F.explode('genre_id')).select('genre_id').toPandas().loc[:, 'genre_id']
all_mapping_data = set(mapping_data.values.tolist())
print all_mapping_data

In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

In [0]:

# Copyright (c) 2020 App Annie Inc. All rights reserved.
"""
Game IQ - Market Size - Pre Aggregation full restatement Cases
"""
import unittest
import re
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df, query
from aadatapipelinecore.core.utils.spark import create_spark



class GenreEstUnifiedData(object):
    """
    Get unified data
    """
    _s3_bucket_path = "s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
                      "store.genre-est.v1/fact/granularity=daily/date={}/"

    def __init__(self, spark):
        self.spark = spark

    def get(self, date):
        return self.spark.read.parquet(self._s3_bucket_path.format(date))


class GenreEstDBData(object):
    """
    Get genre data from DB
    """
    _pg_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
        )
    )

    def get(self, sql):
        """
        :param sql: sql string
        :return: sql result
        :type sql: str
        :rtype: list
        """
        return query(self._pg_dsn, sql)


class DNAGenreIdProductMappingData(object):
    """
    Get mapping data (DNA) from unified level
    """
    _s3_bucket_path = "s3://b2c-prod-data-pipeline-unified-dna/unified/" \
                      "dna.genre_id_product_mapping.v1/dimension/"

    def __init__(self, spark):
        self.spark = spark

    def get(self):
        return self.spark.read.parquet(self._s3_bucket_path)


class DNAMappingDBData(object):
    """
    Get mapping data (DNA) from DB
    """
    _pg_dsn = pg_settings('DNA')

    def get(self, sql):
        """
        :param sql: sql str
        :type sql: str
        :return: mapping data
        :rtype: pandas dataframe
        """
        return query_df(self._pg_dsn, sql)


def get_date_list(start_date, end_date, freq="D"):
    return [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]


def compare_df(df1, df2, on=None):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
        diff_df = diff_df.loc[diff_df["_merge"] != "both"]
        if not diff_df.empty:
            return diff_type, diff_df
        return None, None


class TestGameIQMarketSizeFullRestatement(PipelineTest):
    trigger_date_config = ('* * * * 6', 0)
    
    @classmethod
    def setUpClass(cls):
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext    

    def _compare_genre(self, date):
        """
        :param date: date need to compare
        :type date: str
        :return: genre found in store_genre, but not in dna_mapping
        :rtype: spark.Dataframe
        """
        df_store_genre = GenreEstUnifiedData(self.spark).get(date)
        df_dna_mapping = DNAGenreIdProductMappingData(self.spark).get()
        df_dna_mapping = df_dna_mapping.withColumn("genre_id", F.explode("genre_id"))

        df_all_genre = df_dna_mapping.groupby("genre_id").agg({'genre_id': 'count'}).select('genre_id')
        df_store_genre = df_store_genre.select('genre_id')
        # diff_genre = df_all_genre.subtract(df_store_genre)
        diff_genre = df_store_genre.subtract(df_all_genre)

        return diff_genre

    def test_full_restatement_new(self):
        # ALL MAPPING DATA
        mapping_data = DNAGenreIdProductMappingData(self.spark).get()
        mapping_data = mapping_data.withColumn('genre_id', F.explode('genre_id')).select('genre_id')
        mapping_data = mapping_data.toPandas().loc[:, 'genre_id']
        all_mapping_data = set(mapping_data.values.tolist())
        sql = """
        select device_id, store_id, date, genre_id, modifier_id, download, revenue 
        from plproxy.execute_select_nestloop($proxy$ 
            select device_id, store_id, date, genre_id, modifier_id, download, revenue
            from aa.genre_store_daily_estimate
            where date='{}'
        $proxy$) tbl (device_id SMALLINT, store_id INT, date DATE , 
        genre_id BIGINT, modifier_id BIGINT, download BIGINT, revenue BIGINT);
        """
        all_mapping_data_length = float(len(all_mapping_data))
        date_list = get_date_list('2020-03-04', self.check_date_str)
        for date in date_list:
            db_data = GenreEstDBData().get(sql.format(date))
            if not db_data:
                continue
            db_set = set([])
            for row in db_data:
                db_set.add(row[3])

            diff_len = float(len(all_mapping_data - db_set))
            ratio = diff_len / all_mapping_data_length
            if ratio > 0.15:
                print date, diff_len

unittest.main(argv=[''], verbosity=2)
%tb

In [0]:


log = {
    '2010-12-31': 46,
    '2011-12-31': 71,
    '2012-12-31': 90,
    '2013-12-31': 98,
    '2014-12-31': 108,
    '2015-12-31': 111,
    '2016-12-31': 115,
    '2017-12-31': 119,
    '2018-12-31': 123,
    '2019-12-31': 122
}
def temp_log_to_s3(log, name):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'aa.store.game_iq/{}.json'.format(name))
    # log_str = ""
    # for k, v in log.items():
    #     log_str += k + ',' + str(v) + ';'
    # log_str = log_str[:-1]
    s3object.put(Body=json.dumps(log))

    return json.dumps(log)


def s3_to_temp_log(name):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'aa.store.game_iq/{}.json'.format(name))
    body = s3object.get()['Body'].read()
    print body
    body = json.loads(body)
    # res = body.split(';')
    # res = [i.split(',') for i in res]
    # res = {i: int(j) for i, j in res}

    return body

# def s3_to_temp_log():
#     import boto3
#     s3 = boto3.resource('s3')
#     s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/gameiq/test1.txt')
#     body = s3object.get()['Body'].read()
#     res = body.split(';')
#     res = [i.split(',') for i in res]
#     res = [(i, int(j)) for i, j in res]
#     return res

# print temp_log_to_s3(log, "genre_min_count")
print s3_to_temp_log('genre_min_count')

In [0]:

# Copyright (c) 2020 App Annie Inc. All rights reserved.
"""
Game IQ - Market Size - Pre Aggregation full restatement Cases
"""

import unittest
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.db_check_utils import query_df, query
from conf.settings import *
from aadatapipelinecore.core.utils.spark import create_spark
from applications.db_check_v1.common.utils import get_date_list


class GenreEstDBData(object):
    """
    Get genre data from DB
    """
    _pg_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
        )
    )

    def get(self, sql):
        """
        :param sql: sql string
        :return: sql result
        :type sql: str
        :rtype: list
        """
        return query(self._pg_dsn, sql)


class DNAMappingDBData(object):
    """
    Get mapping data (DNA) from DB
    """
    _pg_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DNA_NAME,
        user=PG_DNA_ACCESS_ID,
        host=PG_DNA_HOSTS[0][0],
        password=PG_DNA_SECRET_KEY,
        port=PG_DNA_HOSTS[0][1]
        )
    )

    def get(self, sql):
        """
        :param sql: sql str
        :type sql: str
        :return: mapping data
        :rtype: pandas dataframe
        """
        return query_df(self._pg_dsn, sql)

def temp_log_to_s3(log, name):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'aa.store.game_iq/{}.json'.format(name))
    s3object.put(Body=json.dumps(log))

    return json.dumps(log)


def s3_to_temp_log(name):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'aa.store.game_iq/{}.json'.format(name))
    body = s3object.get()['Body'].read()
    body = json.loads(body)

    return body

def get_date_and_count(sql, date_list):
    res = {}
    for date in date_list:
        db_data = GenreEstDBData().get(sql.format(date))
        if not db_data:
            continue
        db_set = set([])
        for row in db_data:
            db_set.add(row[3])
        db_set_len = len(db_set)
        res[date[:4]] = db_set_len
    return res

class TestGameIQMarketSizeFullRestatement(PipelineTest):
    trigger_date_config = ('30 8 * * 6', 0)
    @classmethod
    def setUpClass(cls):
        cls.spark = create_spark()
        cls.sc = cls.spark.sparkContext

    def verify_each_day(self, spark, sql, date_list, min_genre_count, threshold=5):
        """
        Regression test:
        To make sure genre_id count in one year is less than this year's last day (12/31),
        and greater than this year's first day (01/01).

        :param threshold:
        To ignore some minor error, there is a threshold (default value is 5):
            LOWER limit - 5 < genre_id count (daily) < UPPER limit + 5

        LOWER limit: Derive from last routine test. (Store in "s3://b2c-prod-data-pipeline-qa/aa.store.game_iq.json
        UPPER limit: All distinct genre_id count from DB.
        """
        mapping_sql = 'SELECT genre_id FROM dna_genre_id_product_mapping'
        mapping_df_raw = DNAMappingDBData().get(mapping_sql)
        spark_df = spark.createDataFrame(mapping_df_raw)
        exploded_spark_df = spark_df.select(explode('genre_id').alias('genre_id'))
        upper_limit = exploded_spark_df.distinct().count()
        for date in date_list:
            db_data = GenreEstDBData().get(sql.format(date))
            if not db_data:
                continue
            db_set = set([])
            for row in db_data:
                db_set.add(row[1])  # genre_id is the 2nd column
            db_set_len = len(db_set)

            # if check date is January 1st, there is no historical data to indicate lower limit,
            # so I use last year lower limit as this year's lower limit.
            if date[:4] in min_genre_count:
                lower_limit = min_genre_count[date[:4]]
            else:
                temp_date = str(int(date[:4]) - 1)  # last year
                print temp_date
                lower_limit = min_genre_count[temp_date]

            self.assertTrue(db_set_len <= upper_limit + threshold,
                            msg="out of UPPER limit... {}, genre id count:, {}, "
                                "expected count: less than {}".format(date, db_set_len, upper_limit))

            self.assertTrue(db_set_len >= lower_limit - threshold,
                            msg="out of LOWER limit... {}, genre id count:, {}, "
                                "expected count: less than {}".format(date, db_set_len, lower_limit))

    def test_full_restatement(self):
        """
        scope :
                dimension completeness : date, genre id
        method:
                check docstring in method "verify_each_day"
        date: regression all date
        """
        sql = '''
            select distinct date, genre_id
            from plproxy.execute_select_nestloop($proxy$
            select distinct date, genre_id
            from aa.genre_store_daily_estimate
            where date = '{}'
            $proxy$) tbl (date DATE, genre_id BIGINT);
            '''
        # Verify each day from 2010-07-04 to check_date
        date_list = get_date_list('2020-03-01', self.check_date_str, freq='D')
        print self.check_date_str
        self.verify_each_day(self.spark, sql, date_list, s3_to_temp_log('genre_min_count'))

        # Add the latest genre_id count to s3 for next db check.
        # SPECIAL_DATE: I use the first day of each year as lower limit,
        # but this data set is from 2010-07-04, so I manually add this special relation
        # to log after test.
        SPECIAL_DATE = ['2010-07-04', 37]
        date_list = get_date_list('2010-01-01', self.check_date_str, freq='YS')
        min_genre_count = get_date_and_count(sql, date_list)
        min_genre_count[SPECIAL_DATE[0][:4]] = SPECIAL_DATE[1]
        temp_log_to_s3(min_genre_count, 'genre_min_count')

        # date_list = get_date_list('2010-12-31', self.check_date_str, freq='Y')
        # # max_genre_count = {'2020': 122, '2019': 123, '2018': 119 ... ...}
        # max_genre_count = get_date_and_count(sql, date_list)
        # temp_log_to_s3(max_genre_count, 'genre_max_count')


unittest.main(argv=[''], verbosity=2)
%tb

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store.game_iq/
aws s3 cp s3://b2c-prod-data-pipeline-qa/aa.store.game_iq/genre_min_count.json -

In [0]:

def temp_log_to_s3(log, name):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'aa.store.game_iq/{}.json'.format(name))
    # log_str = ""
    # for k, v in log.items():
    #     log_str += k + ',' + str(v) + ';'
    # log_str = log_str[:-1]
    s3object.put(Body=json.dumps(log))

    return json.dumps(log)


def s3_to_temp_log(name):
    import boto3
    import json
    s3 = boto3.resource('s3')
    s3object = s3.Object('b2c-prod-data-pipeline-qa', 'aa.store.game_iq/{}.json'.format(name))
    body = s3object.get()['Body'].read()
    print body
    body = json.loads(body)
    # res = body.split(';')
    # res = [i.split(',') for i in res]
    # res = {i: int(j) for i, j in res}

    return body
SPECIAL_DATE = ['2010-07-04', 37]
sql = '''
    select distinct date, genre_id
    from plproxy.execute_select_nestloop($proxy$
    select distinct date, genre_id
    from aa.genre_store_daily_estimate
    where date = '{}'
    $proxy$) tbl (date DATE, genre_id BIGINT);
    '''
date_list = get_date_list('2010-01-01', '2020-04-18', freq='YS')
print date_list
min_genre_count = get_date_and_count(sql, date_list)
min_genre_count[SPECIAL_DATE[0][:4]] = SPECIAL_DATE[1]
print temp_log_to_s3(min_genre_count, 'genre_min_count')

In [0]:

{"2020": 122, "2019": 123, "2018": 119, "2015": 108, "2014": 98, "2017": 115, "2016": 111, "2011": 48, "2010": 37, "2013": 90, "2012": 77}

In [0]:


from conf.settings import PG_DNA_NAME, PG_DNA_ACCESS_ID, PG_DNA_HOSTS, PG_DNA_SECRET_KEY
from applications.db_check_v1.common.db_check_utils import query_df
from pyspark.sql.functions import explode

dna_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DNA_NAME,
        user=PG_DNA_ACCESS_ID,
        host=PG_DNA_HOSTS[0][0],
        password=PG_DNA_SECRET_KEY,
        port=PG_DNA_HOSTS[0][1]
    )
)
# mapping_df_unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.genre_id_product_mapping.v1/dimension/").toPandas()
sql = 'SELECT genre_id FROM dna_genre_id_product_mapping'
mapping_df_raw = query_df(dna_dsn, sql)
spark_df = spark.createDataFrame(mapping_df_raw)
spark_df.show(3)
# mapping_df_raw = mapping_df_raw.explode('genre_id')
exploded_spark_df = spark_df.select(explode('genre_id').alias('genre_id'))
genre_count = exploded_spark_df.distinct().count()

# mapping_df_raw['created_time'] = mapping_df_raw['created_time'].dt.strftime('%Y-%m-%d')
# mapping_df_raw['last_updated_time'] = mapping_df_raw['last_updated_time'].dt.strftime('%Y-%m-%d')
# mapping_df_raw['genre_id'] = [str(map(int, l))  if l else 'None' for l in mapping_df_raw['genre_id']]
# mapping_df_raw['modifier_id'] = [str(map(int, l)) if l else 'None' for l in mapping_df_raw['modifier_id']]

# mapping_df_unified['genre_id'] = mapping_df_unified['genre_id'].astype("str")
# mapping_df_unified['modifier_id'] = mapping_df_unified['modifier_id'].astype("str")
# print mapping_df_raw
# # print mapping_df_unified

# def _compare_df(df1, df2, on=None):
#     for diff_type in ["left", "right"]:
#         diff_df = df1.merge(df2, indicator=True, how=diff_type, on=on)  # .loc[lambda x : x['_merge']!='both']
#         diff_df = diff_df.loc[diff_df["_merge"] != "both"]
#         print diff_df.empty
#         if len(diff_df) != 0:
#             print diff_type
#             print diff_df


# # # print mapping_df_raw.genre_id.dtypes
# # # print '*'*100
# # # print mapping_df_unified.genre_id.dtypes
# # # print mapping_df_raw
# # # print mapping_df_unified

# _compare_df(mapping_df_raw, mapping_df_unified, on=["product_id", "genre_id"]) # "modifier_id","last_updated_time",  "created_time", "created_by", "last_updated_by", "comments", 
# print "pass"

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
select * from public.in_app_dna_info_mapping limit 3;
EOF

In [0]:
%%sh
